<a href="https://colab.research.google.com/github/JaeminKIM-Irene/FC_AI_ML_Project/blob/main/irenekim/ML_Project_LightGBM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install lightgbm optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.model_selection import KFold

import optuna

from sklearn.model_selection import StratifiedKFold
from functools import partial

from sklearn.metrics import roc_auc_score

In [3]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# read data
result_list = []
base_url = '/content/drive/MyDrive/Colab Notebooks/data/playground-series-s3e23/'
train = pd.read_csv(base_url+'train.csv', index_col='id')
test = pd.read_csv(base_url+'test.csv', index_col='id')
submission = pd.read_csv(base_url+'sample_submission.csv', index_col='id')

with pd.option_context('display.min_rows', 6) :
  display(train)

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
id,,,,,,,,,,,,,,,,,,,,,
0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,5448.79,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,936.71,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,1754.01,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101760,26.0,1.0,1.0,1.0,83.0,360.17,0.04,22.75,12.56,5893.69,...,20,0,4,0,10.0,8.0,51.0,28.0,1.0,False
101761,10.0,2.0,1.0,2.0,43.0,191.76,0.15,6.46,29.53,1534.04,...,7,0,1,0,11.0,14.0,24.0,19.0,3.0,False
101762,136.0,18.0,18.0,1.0,296.0,1704.57,0.02,44.82,44.08,77011.02,...,102,1,11,0,22.0,31.0,162.0,135.0,35.0,False


In [ ]:
# PCA

In [5]:
# data split for baseline model
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=61)

In [6]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(96674, 21) (96674,) (5089, 21) (5089,)


## Baseline Model

In [ ]:
model = LGBMClassifier()

model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 21922, number of negative: 74752
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3581
[LightGBM] [Info] Number of data points in the train set: 96674, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226762 -> initscore=-1.226685
[LightGBM] [Info] Start training from score -1.226685


LGBMClassifier()

In [ ]:
print('======Prediction======')
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)

train_score = roc_auc_score(y_train, pred_train)
val_score = roc_auc_score(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Test Score : %.4f" % val_score)

======Prediction======
Train Score : 0.6760
Test Score : 0.6630


## Optuna

In [21]:
def optimizer(trial, X, y, K) :
  #num_leaves = trial.suggest_categorical('num_leaves', [64, 128, 256, 512])
  num_leaves=64
  max_depth = trial.suggest_int('max_depth', 10, 25)
  #learning_rate = trial.suggest_float('learning_rate', 0.001, 0.3)
  learning_rate=0.02051093484555017
  min_child_samples = trial.suggest_int('min_child_samples', 1, 100)
  #colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 0.8)
  colsample_bytree = 0.6103281689909603

  model = LGBMClassifier(num_leaves=num_leaves,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         min_child_samples=min_child_samples,
                         colsample_bytree=colsample_bytree,
                         is_unbalance=True,
                         random_state=61)

  folds = StratifiedKFold(shuffle=True, random_state=61)
  oof = np.full(len(train), np.nan)
  auc_list = []

  for train_idx, val_idx in folds.split(X, y) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]

    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    model.fit(X_train, y_train)

    try :
      y_val_pred = model.predict_proba(X_val)[:, 1]
    except :
      y_val_pred = model.decision_function(X_val)
    oof[val_idx] = y_val_pred
    auc = roc_auc_score(y_val, y_val_pred)
    auc_list.append(auc)

  return np.mean(auc_list)

In [22]:
K = 5
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)

study = optuna.create_study(direction='maximize')
study.optimize(opt_func, n_trials=50)

[I 2023-10-16 07:06:55,275] A new study created in memory with name: no-name-f1734636-1da8-42ad-a540-7f28ba0ecfbd


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018107 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018521 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3553
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[Light

[I 2023-10-16 07:07:06,550] Trial 0 finished with value: 0.7918694660200856 and parameters: {'max_depth': 22, 'min_child_samples': 6}. Best is trial 0 with value: 0.7918694660200856.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018180 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015858 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3553
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[Light

[I 2023-10-16 07:07:17,591] Trial 1 finished with value: 0.7923210732192756 and parameters: {'max_depth': 10, 'min_child_samples': 99}. Best is trial 1 with value: 0.7923210732192756.


...

[I 2023-10-16 07:15:45,378] Trial 47 finished with value: 0.7922318378922715 and parameters: {'max_depth': 11, 'min_child_samples': 88}. Best is trial 1 with value: 0.7923210732192756.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016322 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3553
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[Light

[I 2023-10-16 07:15:56,705] Trial 48 finished with value: 0.7921274746721624 and parameters: {'max_depth': 13, 'min_child_samples': 55}. Best is trial 1 with value: 0.7923210732192756.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015840 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015967 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3553
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[Light

[I 2023-10-16 07:16:07,770] Trial 49 finished with value: 0.7922541845529576 and parameters: {'max_depth': 12, 'min_child_samples': 95}. Best is trial 1 with value: 0.7923210732192756.


In [ ]:
# boosting type = dart

In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bytree,params_learning_rate,params_max_depth,params_min_child_samples,params_num_leaves,state
0,0,0.785245,2023-10-13 08:23:44.196514,2023-10-13 08:23:53.934372,0 days 00:00:09.737858,0.633648,0.144228,18,53,128,COMPLETE
1,1,0.789403,2023-10-13 08:23:53.936235,2023-10-13 08:24:05.442080,0 days 00:00:11.505845,0.654292,0.080506,15,40,128,COMPLETE
2,2,0.781771,2023-10-13 08:24:05.444761,2023-10-13 08:24:16.280211,0 days 00:00:10.835450,0.537578,0.197202,23,66,128,COMPLETE
3,3,0.787310,2023-10-13 08:24:16.282926,2023-10-13 08:24:27.371523,0 days 00:00:11.088597,0.557604,0.109677,13,83,256,COMPLETE
4,4,0.776238,2023-10-13 08:24:27.374829,2023-10-13 08:24:34.843898,0 days 00:00:07.469069,0.595357,0.294617,25,19,64,COMPLETE
5,5,0.787429,2023-10-13 08:24:34.845878,2023-10-13 08:24:51.671386,0 days 00:00:16.825508,0.777333,0.050287,19,41,256,COMPLETE
6,6,0.785743,2023-10-13 08:24:51.673249,2023-10-13 08:25:07.272607,0 days 00:00:15.599358,0.594577,0.084091,11,17,512,COMPLETE
7,7,0.787909,2023-10-13 08:25:07.277299,2023-10-13 08:25:18.383862,0 days 00:00:11.106563,0.674531,0.137248,11,87,128,COMPLETE
8,8,0.782927,2023-10-13 08:25:18.385853,2023-10-13 08:25:31.420240,0 days 00:00:13.034387,0.516427,0.093160,22,10,256,COMPLETE
9,9,0.791478,2023-10-13 08:25:31.422203,2023-10-13 08:25:40.419312,0 days 00:00:08.997109,0.763040,0.058880,19,11,64,COMPLETE


In [11]:
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7921
Best Params:  {'num_leaves': 64, 'max_depth': 10, 'learning_rate': 0.02051093484555017, 'min_child_samples': 89, 'colsample_bytree': 0.7680093543964613}


In [15]:
# fix learning rate 0.02051093484555017
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7923
Best Params:  {'num_leaves': 64, 'max_depth': 10, 'min_child_samples': 75, 'colsample_bytree': 0.6869212128506842}


In [19]:
# fix learning rate 0.02051093484555017, num_leaves 64
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7923
Best Params:  {'max_depth': 24, 'min_child_samples': 81, 'colsample_bytree': 0.6103281689909603}


In [23]:
# fix learning rate 0.02051093484555017, num_leaves 64, colsample_bytree 0.6103281689909603
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7923
Best Params:  {'max_depth': 10, 'min_child_samples': 99}


In [ ]:
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7923
Best Params:  {'num_leaves': 64, 'max_depth': 14, 'learning_rate': 0.04151372498765896, 'min_child_samples': 64, 'colsample_bytree': 0.7039329213868132}


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [20]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

## Test & Submission file

In [ ]:
def oof_preds(best_model) :
  folds = StratifiedKFold(shuffle=True, random_state=61)
  final_preds = []
  auc_list = []

  for i, (train_idx, val_idx) in enumerate(folds.split(X, y)) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]
    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    print(f"========== Fold {i+1} ==========")
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]
    test_preds = best_model.predict_proba(test)[:, 1]
    final_preds.append(test_preds)
    auc = roc_auc_score(y_val, preds)

    auc_list.append(auc)

  print(f'AUC :', np.mean(auc))
  return final_preds

In [ ]:
best_params = study.best_trial.params

best_model = LGBMClassifier(**best_params,
                            random_state=61)

preds = oof_preds(best_model)
preds = np.mean(preds, axis=0)

preds

========== Fold 1 ==========
[LightGBM] [Info] Number of positive: 18451, number of negative: 62959
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019327 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3547
[LightGBM] [Info] Number of data points in the train set: 81410, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226643 -> initscore=-1.227365
[LightGBM] [Info] Start training from score -1.227365
========== Fold 2 ==========
[LightGBM] [Info] Number of positive: 18451, number of negative: 62959
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017872 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3546
[LightGBM] [Info] Number of data points in the train set: 81410, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226643 -> initscore=-1.227365
[L

array([0.21875383, 0.18928986, 0.64816721, ..., 0.17000563, 0.10214024,
       0.80084926])

In [ ]:
submission.defects = preds

In [ ]:
submission.to_csv('submission_lightGBM_allparameters.csv')